## Data Source:
https://docs.google.com/spreadsheets/d/16Oi7F6RZh6w1mSO821RW6UzqWE-oSExZDHLhRzcK5Ug/edit?fbclid=IwAR0zF8xkZYIBzj74aLYKc5SvkIGXey5OY1DlSVOSducLX3fnLaeMY2wzX8w#gid=2028039576

## install chart_studio

In [ ]:
%%sh
pip install chart_studio

In [2]:
from chart_studio.plotly import plot, iplot

In [3]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objects as go
 

daily_stats = pd.read_csv('https://od.cdc.gov.tw/eic/Day_Confirmation_Age_County_Gender_19CoV.csv')
daily_stats['個案研判日_datetime'] = pd.to_datetime(daily_stats['個案研判日']).dt.date

daily_stats.rename(columns= {'個案研判日':'個案研判日_string'}, inplace=True)

daily_stats['年齡層'] = daily_stats['年齡層'].replace({'0':'00-04',
                           '1':'00-04',
                           '2':'00-04',
                           '3':'00-04',
                           '4':'00-04',
                           '5-9':'05-09'})

daily_stats['年齡層'] = daily_stats['年齡層'].str.replace('-', '~')

date_start = datetime.date(2022, 4, 1)
mask = (daily_stats.個案研判日_datetime > date_start) & (daily_stats.確定病名 == '嚴重特殊傳染性肺炎')

daily_stats = daily_stats[mask]

daily_stats['是否為境外移入'].replace('None', '否', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [4]:
daily_stats

,確定病名,個案研判日_string,縣市,鄉鎮,性別,是否為境外移入,年齡層,確定病例數,個案研判日_datetime
15268,嚴重特殊傳染性肺炎,2022/04/02,台中市,大里區,女,否,35~39,1,2022-04-02
15269,嚴重特殊傳染性肺炎,2022/04/02,台中市,大里區,男,否,00~04,1,2022-04-02
15270,嚴重特殊傳染性肺炎,2022/04/02,台中市,南屯區,男,否,65~69,1,2022-04-02
15271,嚴重特殊傳染性肺炎,2022/04/02,台北市,士林區,女,否,25~29,1,2022-04-02
15272,嚴重特殊傳染性肺炎,2022/04/02,台北市,士林區,女,否,35~39,1,2022-04-02
...,...,...,...,...,...,...,...,...,...
490679,嚴重特殊傳染性肺炎,2022/06/21,澎湖縣,湖西鄉,男,否,40~44,1,2022-06-21
490680,嚴重特殊傳染性肺炎,2022/06/21,澎湖縣,湖西鄉,男,否,45~49,1,2022-06-21
490681,嚴重特殊傳染性肺炎,2022/06/21,澎湖縣,湖西鄉,男,否,55~59,2,2022-06-21
490682,嚴重特殊傳染性肺炎,2022/06/21,澎湖縣,湖西鄉,男,否,65~69,1,2022-06-21


In [5]:
# 4月起，各縣市每日確診趨勢

pivot1 = daily_stats.pivot_table(index=['個案研判日_string','縣市'], values='確定病例數', aggfunc='sum')
pivot1 = pivot1.reset_index()
mask = pivot1.縣市 != '空值'
pivot1 = pivot1[mask]

fig1 = px.bar(pivot1, y='縣市', x='確定病例數',
         orientation='h',
         animation_frame='個案研判日_string',
         title='縣市確診病例數')
fig1.update_layout(yaxis_categoryorder = 'total ascending')

fig1.show()

In [6]:
# 境外移入與比例

from plotly.subplots import make_subplots

pivot2 = daily_stats.pivot_table(index='個案研判日_datetime', columns='是否為境外移入', values='確定病例數', aggfunc=sum)
pivot2 = pivot2.reset_index()

pivot2['境外移入比例(%)'] = pivot2.eval('是 / (是+否)*100')

pivot2

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(go.Bar(name='境外移入案例', x=pivot2['個案研判日_datetime'], y=pivot2['是']),
        secondary_y=False)


fig2.add_trace(
  go.Scatter(name='境外移入比例(%)', x=pivot2['個案研判日_datetime'], y=pivot2['境外移入比例(%)']),
  secondary_y=True)

fig2.update_layout(title = '境外移入案例與境外移入比例', yaxis_range=[0, 250])
fig2.update_yaxes(range=[0, 50], secondary_y=True)

fig2.show()

In [7]:
pivot2

是否為境外移入,個案研判日_datetime,否,是,境外移入比例(%)
0,2022-04-02,194,218,52.912621
1,2022-04-03,138,129,48.314607
2,2022-04-04,207,83,28.620690
3,2022-04-05,254,82,24.404762
4,2022-04-06,365,123,25.204918
...,...,...,...,...
76,2022-06-17,53606,64,0.119247
77,2022-06-18,50544,75,0.148166
78,2022-06-19,35589,36,0.101053
79,2022-06-20,56311,65,0.115297


In [8]:
# 本土七日均線

mask = daily_stats.是否為境外移入 == '否'
local_daily = daily_stats[mask]
pivot3 = local_daily.pivot_table(index='個案研判日_datetime', values=['確定病例數'], aggfunc='sum')

pivot3['本土七日均線'] = pivot3.rolling(window=7, min_periods=1).mean().round(2)

pivot3

fig3 = go.Figure(data=[
  go.Bar(name='本土案例', x=pivot3.index, y=pivot3['確定病例數']),
  go.Scatter(name='本土七日均線', x=pivot3.index, y=pivot3['本土七日均線'])
])
fig3.update_layout(title = '本土七日均線')

fig3.show()


In [9]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_datetime','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

#test_df = pivot4[ pivot4['個案研判日_datetime'] == datetime.date(2022, 4, 2) ]


male  = pivot4[pivot4['性別'] == '男']
female = pivot4[pivot4['性別'] == '女']

layout = go.Layout(yaxis=go.layout.YAxis(title='年齡'))

fig4 = go.Figure(data=[
    go.Bar(name='男',
        y=male['年齡層'],
        x=male['確定病例數'],
        orientation='h'),

    go.Bar(name='女',
        y=female['年齡層'],
        x=female['確定病例數'].apply(lambda x: -x),
        orientation='h')
])

fig4.update_layout(barmode='relative',
         title='確診年齡層分布-1')


fig4.show()


In [10]:

# # 確診年齡層

# pivot4 = daily_stats.pivot_table(index=['個案研判日_datetime','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
# pivot4 = pivot4.reset_index()
# #pivot4['確定病例數']
# #pivot4.index
# #pivot4.query('個案研判日_datetime > datetime.date(2022,4,10)')

# #fig4 = go.Figure()
# test_df = pivot4[ pivot4['個案研判日_datetime'] == datetime.date(2022, 4, 2) ]

# #fig4 = px.bar(test_df[test_df['性別']=='男'], x="年齡層", y='確定病例數', barmode='overlay', color='性別')

# male  = test_df[test_df['性別'] == '男']
# female = test_df[test_df['性別'] == '女']

# layout = go.Layout(yaxis=go.layout.YAxis(title='年齡'))

# fig4 = go.Figure(data=[
#     go.Bar(name='男',
#         y=male['年齡層'],
#         x=male['確定病例數'],
#         orientation='h'),

#     go.Bar(name='女',
#         y=female['年齡層'],
#         x=female['確定病例數'].apply(lambda x: -x),
#         orientation='h')
# ])

# #daily_stats['年齡層'] = daily_stats['年齡層'].str.replace({'0':'0-4',

# fig4.update_layout(barmode='overlay')


# fig4.show()
# #pivot4


In [11]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_string','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

fig5 = px.bar(pivot4, x="年齡層", y='確定病例數', barmode='group', color='確定病例數',
              facet_row='性別',
              hover_data=['個案研判日_string','性別','年齡層','確定病例數'],
              title='確診年齡層分布-2')




fig5.show()


In [12]:
# 確診年齡層

pivot4 = daily_stats.pivot_table(index=['個案研判日_string','年齡層','性別'], values=['確定病例數'], aggfunc='sum')
pivot4 = pivot4.reset_index()

fig6 = px.bar(pivot4, x="年齡層", y='確定病例數', barmode='group', color='確定病例數',
              facet_row='性別',
              hover_data=['個案研判日_string','性別','年齡層','確定病例數'],
              title='確診年齡層分布-3',
              animation_frame='個案研判日_string')




fig6.show()


In [13]:
%%sh
pip install jupyter-dash -q
pip install dash-cytoscape -q

In [14]:
from jupyter_dash import JupyterDash  # pip install dash
import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
from dash import dcc
from dash.dependencies import Output, Input

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [15]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Covid-19 Analysis'),

    html.Div(children=dcc.Graph(id='fig1', figure=fig1)),
    html.Div(children=dcc.Graph(id='fig2', figure=fig2)),
    html.Div(children=dcc.Graph(id='fig3', figure=fig3)),
    html.Div(children=dcc.Graph(id='fig4', figure=fig4)),
    html.Div(children=dcc.Graph(id='fig5', figure=fig5)),
    html.Div(children=dcc.Graph(id='fig6', figure=fig6)),

    

    

    
])

if __name__ == '__main__':
    app.run_server(debug=True)
    app.run_server(mode='inline', port=8030)

Dash app running on:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Resources
dash app gallery
1. https://dash.gallery/Portal/

dash-pivottable
1. https://github.com/plotly/dash-pivottable

2. https://dash.gallery/dash-pivottable/